# Data Gathering for Indoor Localization

This notebook contains the process of loading a text file containing indoor localization data into a dataframe. [Will convert this into a .yp file later]

### Problem Description

> **Task**: Search for a Survey paper and list some approaches present in the literature 
https://arxiv.org/pdf/1709.01015
In this paper, the following localization techniques are listed:
* Angle of Arrival(AoA): Detects angle of signal using multiple antennas
* Time of Flight(ToF): Measures time a signal takes to arrive using signal propagation speed
* Return Time of Flight(RToF): ToF, but also consideres time it takes back to sender after reaching receiver. uses a reply mechanism
* Received Signal Strength(RSS): Uses strenght of signal to estimate distance using known signal maps (fingerprinting)

based on:
* Wifi
* Radio Frequency Identification Device(RFID)
* Ultra Wideband
* Bluetooth

> **Task**: Formulate the Problem Description, Challanges and Ml formulation

**Problem Description**: The goal of this project to create a indoor localization estimator based on KNN using Wifi Signal Strength data derived from [this dataset of the universitiy of mannheim](https://ieee-dataport.org/open-access/crawdad-mannheimcompass-v-2006-09-13). In this dataset, the system estimates the position of mobile devices using RSS values collected from nearby access points. The task of the indoor localization estimator is to estimate the X, Y coordinates of a device as well as the degree.

**Challenges**:
* Accuracy is may vary due to various factors like noise or user orientation/degree
* High Dimensionality of the data may make it hard to interpret the scans
* Uneven data distribution, as the amount of points collected per access point may vary
* Environemt in which data is collected may vary over long time, which can also affect the accuracy

**ML Formulation**:

KNN for estimating the position of a scanned device. For training, the offline data is used and for testing, the online data is used.

Input: RSSI of access points, frequency?

Output: position in coordinates -> estimation of where device is.

### Reading the data

In [1]:
# Includes
from datetime import datetime
import re
import pandas as pd

First, let's take a look at first few lines of the data to get an overview.

In [2]:
offline_file = "data/offline.final.trace.txt"

count = 0
with open(offline_file, 'r') as file:
    for line in file:
        print(line.strip())
        count += 1
        if count == 120:
            break

# timestamp=2006-02-11 08:31:58
# usec=250
# minReadings=110
t=1139643118358;id=00:02:2D:21:0F:33;pos=0.0,0.0,0.0;degree=0.0;00:14:bf:b1:97:8a=-38,2437000000,3;00:14:bf:b1:97:90=-56,2427000000,3;00:0f:a3:39:e1:c0=-53,2462000000,3;00:14:bf:b1:97:8d=-65,2442000000,3;00:14:bf:b1:97:81=-65,2422000000,3;00:14:bf:3b:c7:c6=-66,2432000000,3;00:0f:a3:39:dd:cd=-75,2412000000,3;00:0f:a3:39:e0:4b=-78,2462000000,3;00:0f:a3:39:e2:10=-87,2437000000,3;02:64:fb:68:52:e6=-88,2447000000,1;02:00:42:55:31:00=-84,2457000000,1
t=1139643118744;id=00:02:2D:21:0F:33;pos=0.0,0.0,0.0;degree=0.0;00:14:bf:b1:97:8a=-38,2437000000,3;00:0f:a3:39:e1:c0=-54,2462000000,3;00:14:bf:b1:97:90=-56,2427000000,3;00:14:bf:3b:c7:c6=-67,2432000000,3;00:14:bf:b1:97:81=-66,2422000000,3;00:14:bf:b1:97:8d=-70,2442000000,3;00:0f:a3:39:e0:4b=-79,2462000000,3;00:0f:a3:39:dd:cd=-73,2412000000,3;00:0f:a3:39:e2:10=-83,2437000000,3;02:00:42:55:31:00=-85,2457000000,1
t=1139643119002;id=00:02:2D:21:0F:33;pos=0.0,0.0,0.0;degree=0.0;00:14:bf:b1:

It can be seen, that there is a summary of the timestamp using comments with '#', usec and min amount of readings during that time. These values can be derived from the actual data anyway, therefore they are just skipped. It can be seen that after every 110 datapoints, a comment is present, which indicates that the data is consistent, at least from the first look.


> *Task*: Consider the two following obvious choices of how to store the data in a structured way and give at least a pro and a con for each:
>1. transfer each row in input file to a row in data frame
>2. store one signal per row. That implies that each line in the input file turns into multiple rows in the data frame.

1. transfer each row in input file to a row in data frame
* Pro: Easier to implement
* Con: Hard to query data from each row, as the calculation of the values needs to be repeated every time.

2. store one signal per row. That implies that each line in the input file turns into multiple rows in the data frame.
* Pro: Cleaner to read and query data as every line contains a single value
* Cons: More complex to read data from file

Now each line of the actual data contains the following features:

* t = unix Timestamp (int)

* id = MAC of scanning device (string)

* pos = Position of the scanning device (tuple containing three values, should be split into seperate columns)

* degree = Rotational orientation of the device. from 0° to 360° obviously (float)

* Readings: for each read datapoint repeat:
  * MAC = MAC address of the scanned device (string)
  * RSSI = Signal strength of the scanned device in dBm(int)
  * frequency = Signal frequency of the scanned device in Hz (integer)
  * mode = type of device that sent signal [access point = 3, device in adhoc mode = 1] (intger)

For each line, the different features can easily be extracted using regular expressions. The following code snippet contains the different expressions used for each feature:

In [3]:
regex_t = "t=[0-9]+;" # matches t=<number>;
regex_id = "id=([0-9A-Fa-f]{2}(:[0-9A-Fa-f]{2}){5});" # matches only valid mac addresses
regex_pos = "pos=\d+\.\d+,\d+\.\d+,\d+\.\d+;" # matches x,y,z position, no negative values right now, will add if needed
regex_degree = "degree=(?:360(?:\.0+)?|(?:[0-9]|[1-9][0-9]|1[0-9]{2}|2[0-9]{2}|3[0-5][0-9])(?:\.\d+)?);" # matches degrees from 0 to 360, including precision
regex_readings = "([0-9A-Fa-f]{2}(:[0-9A-Fa-f]{2}){5})=-?\d+(?:\.\d+)?,-?\d+,\d" # matches a single distinct reading, works for the list of all readings in a line.

Now two functions need to be implemented: one for processing a line and one for processing the whole file using the function for processing the line. 
Keep in mind that when matching the regular expression, no additional checks need to be done to ensure that split lines contain the correct amount of values. also, datatypes are not defined here, this will be done when creating the dataframe

In [152]:
def process_line(line: str):
    """
    Processes a line of the localization dataset

    Parameters:
    line (string): A line containing localization measurement data

    Returns:
    A dictionary containing all values with converted datatype.
    """
    line_data = {}

    # t
    match = re.search(regex_t, line)
    if match:
        value = match.group(0).split('=')[1].strip(';')
        line_data["t"] = int(value)

    # id
    match = re.search(regex_id, line)
    if match:
        value = match.group(0).split('=')[1].strip(';')
        line_data["id"] = value

    # pos
    match = re.search(regex_pos, line)
    if match:
        value = match.group(0).split('=')[1].strip(';').split(',') # coordinates are split again by "," seperator
        line_data["pos_x"] = float(value[0])
        line_data["pos_y"] = float(value[1])
        line_data["pos_z"] = float(value[2])

    # degree
    match = re.search(regex_degree, line)
    if match:
        value = match.group(0).split('=')[1].strip(';')
        line_data["degree"] = float(value)

    # readings
    matches = re.finditer(regex_readings, line)
    line_data["readings"] = []
    for match in matches:
        reading = {}
        values = match.group(0).split(',') # this does not split MAC and RSSI, they are seperated by "="
        mac_rssi = values[0].split('=')
        reading["mac"] = mac_rssi[0]
        reading["rssi"] = int(mac_rssi[1])
        reading["frequency"] = int(values[1])
        reading["mode"] = int(values[2])

        line_data["readings"].append(reading)
    return line_data

Due to the fact that the orientation values are not always multiples of 45°, they need to be rounded. in order to do this properly, a functon needs to be implemented:

In [5]:
def round_orientation(angle):
    """
    Rounds an orientation angle to a multiple of 45°

    Parameters:
    line (string): any angle

    Returns:
    rounded orientation as multiple of 45°
    """
    angle = angle % 360 # modulo 360 caps value between 0 and 360
    rounded = round(angle / 45) * 45
    return rounded % 360 # another modulo to round values near 360 to 0

In [227]:
def read_data(file_name: str, submacs: list):
    """
    Reads and converts a localization data file into a dataframe

    Parameters:
    file_name (string): A file containing localization measurement data

    Returns:
    A dataframe containing the processed and cleaned localization data
    """
    rows = []
    with open(file_name, 'r') as file:
        for line in file:     
            if not line.startswith("#"): # to skip commments
                line_dict = process_line(line.strip())
                
                reading_rows = []
                for reading in line_dict['readings']:
                    row = {
                        't': line_dict['t'],
                        'id': line_dict['id'],
                        'pos_x': line_dict['pos_x'],
                        'pos_y': line_dict['pos_y'],
                        'pos_z': line_dict['pos_z'],
                        'degree': line_dict['degree'],
                        'mac': reading['mac'],
                        'rssi': reading['rssi'],
                        'frequency': reading['frequency'],
                        'mode': reading['mode']
                    }
                    reading_rows.append(row)
                rows.extend(reading_rows)
    
    df = pd.DataFrame(rows)
    # convert datatypes
    df = df.astype({ # convert datatypes
        't': 'int64',
        'id': 'string',
        'pos_x': 'float64',
        'pos_y': 'float64',
        'pos_z': 'float64',
        'degree': 'float64',
        'rssi': 'int32',
        'frequency': 'int64',
        'mode': 'int8'
    })

    # add angle column with rounded degrees
    df['angle'] = df['degree'].apply(round_orientation)
    # skip rows with mode != 3 (no adhc mode)
    df = df[df['mode'] == 3]
    # remove mode = type  column
    df = df.drop('mode', axis=1)
    # Format time
    df['time'] = pd.to_datetime(df['t'], unit='ms') # unix timestamp has ms format
    df = df.rename(columns={'t': 'raw_time'})
    # remove z coordinate
    df = df.drop('pos_z', axis=1)
    # drop frequency 
    df = df.drop('frequency', axis=1)
    # remove id = scanMAC
    df = df.drop('id', axis=1) # there was only one mac address, this column is not needed
    # filter dataframe to contain only values with specified mac addresses
    df = df[df['mac'].isin(submacs)]
    # create posXY locatio identifier like = 0.0-0.0
    df['posXY'] = df["pos_x"].astype(str) + "-" + df["pos_y"].astype(str)
    return df

In [228]:
offline_file = "data/offline.final.trace.txt"
submacs = ["00:0f:a3:39:e1:c0", "00:0f:a3:39:dd:cd", "00:14:bf:b1:97:8a", "00:14:bf:3b:c7:c6", "00:14:bf:b1:97:90", "00:14:bf:b1:97:8d", "00:14:bf:b1:97:81"]

df_offline = read_data(offline_file, submacs)
df_offline = df_offline.drop_duplicates() # there are a few thousand duplicate rows in the data, therefore they are remove
df_offline

,raw_time,pos_x,pos_y,degree,mac,rssi,angle,time,posXY
0,1139643118358,0.0,0.0,0.0,00:14:bf:b1:97:8a,-38,0,2006-02-11 07:31:58.358,0.0-0.0
1,1139643118358,0.0,0.0,0.0,00:14:bf:b1:97:90,-56,0,2006-02-11 07:31:58.358,0.0-0.0
2,1139643118358,0.0,0.0,0.0,00:0f:a3:39:e1:c0,-53,0,2006-02-11 07:31:58.358,0.0-0.0
3,1139643118358,0.0,0.0,0.0,00:14:bf:b1:97:8d,-65,0,2006-02-11 07:31:58.358,0.0-0.0
4,1139643118358,0.0,0.0,0.0,00:14:bf:b1:97:81,-65,0,2006-02-11 07:31:58.358,0.0-0.0
...,...,...,...,...,...,...,...,...,...
1181622,1141936870456,20.0,8.0,315.7,00:14:bf:b1:97:8d,-61,315,2006-03-09 20:41:10.456,20.0-8.0
1181623,1141936870456,20.0,8.0,315.7,00:14:bf:b1:97:8a,-60,315,2006-03-09 20:41:10.456,20.0-8.0
1181624,1141936870456,20.0,8.0,315.7,00:14:bf:b1:97:81,-60,315,2006-03-09 20:41:10.456,20.0-8.0
1181625,1141936870456,20.0,8.0,315.7,00:14:bf:b1:97:90,-68,315,2006-03-09 20:41:10.456,20.0-8.0


In [ ]:
df_pkl = pd.read_pickle('data/df.pkl')
df_pkl
# notes for comparison: i have more values overall and frequency column is not part of the .pkl file so i also removed it

,time,posX,posY,orientation,mac,signal,angle,rawTime,posXY
0,2006-02-11 07:31:58.358,0.0,0.0,0.0,00:14:bf:b1:97:8a,-38,0,1139643118358,0.0-0.0
1,2006-02-11 07:31:58.358,0.0,0.0,0.0,00:14:bf:b1:97:90,-56,0,1139643118358,0.0-0.0
2,2006-02-11 07:31:58.358,0.0,0.0,0.0,00:0f:a3:39:e1:c0,-53,0,1139643118358,0.0-0.0
3,2006-02-11 07:31:58.358,0.0,0.0,0.0,00:14:bf:b1:97:8d,-65,0,1139643118358,0.0-0.0
4,2006-02-11 07:31:58.358,0.0,0.0,0.0,00:14:bf:b1:97:81,-65,0,1139643118358,0.0-0.0
...,...,...,...,...,...,...,...,...,...
1181622,2006-03-09 20:41:10.456,20.0,8.0,315.7,00:14:bf:b1:97:8d,-61,315,1141936870456,20.0-8.0
1181623,2006-03-09 20:41:10.456,20.0,8.0,315.7,00:14:bf:b1:97:8a,-60,315,1141936870456,20.0-8.0
1181624,2006-03-09 20:41:10.456,20.0,8.0,315.7,00:14:bf:b1:97:81,-60,315,1141936870456,20.0-8.0
1181625,2006-03-09 20:41:10.456,20.0,8.0,315.7,00:14:bf:b1:97:90,-68,315,1141936870456,20.0-8.0


In [59]:
print("Check if they are equal: " , df_pkl.equals(df_offline))

Check if they are equal:  False


In [180]:
diff_indices = df_offline.index.difference(df_pkl.index)
for i in diff_indices:
    print(i)
print("len ", len(diff_indices))

319
336
360
392
455
572
807
1010
1205
1495
1582
1945
2403
2946
3095
3568
4039
4059
4367
4671
4967
5075
5151
5525
6021
6427
6722
7530
7678
8561
8732
9128
9135
9322
9953
9970
11293
11901
11980
12527
12560
12814
13076
13143
13167
13388
13407
13965
14237
14320
14483
14501
15128
15136
15269
15295
15425
15671
15695
15719
15736
15760
15768
15802
15817
15825
15839
15878
15887
15903
15938
15956
15962
15971
15980
15987
16042
16142
16176
16272
16359
16409
16421
16456
16464
16521
18608
18616
18664
19194
19200
19302
19400
19486
19521
19563
19832
19880
20394
20526
20611
20666
20867
20934
20966
20984
21048
21084
21371
21732
22004
22078
22266
22631
22676
23015
23124
23142
23271
23341
23451
23494
23559
23727
23735
23833
23914
24550
24590
24799
24834
24905
25345
25381
25467
25780
25943
26035
26189
26214
26630
26857
26904
26909
26921
27012
27133
27137
27326
27376
27433
27491
27596
28685
29472
29777
29785
30886
31036
31110
31306
31390
31408
31549
31626
31644
31697
31719
32017
32167
32332
32751
33245
33371

In [234]:
df_offline.loc[1181538]

raw_time                 1141936867432
pos_x                             20.0
pos_y                              8.0
degree                           315.7
mac                  00:14:bf:b1:97:90
rssi                               -69
angle                              315
time        2006-03-09 20:41:07.432000
posXY                         20.0-8.0
Name: 1181538, dtype: object

In [233]:
df_pkl.loc[1181538]

time           2006-03-09 20:41:07.432000
posX                                 20.0
posY                                  8.0
orientation                         315.7
mac                     00:14:bf:b1:97:90
signal                                -69
angle                                 315
rawTime                     1141936867432
posXY                            20.0-8.0
Name: 1181538, dtype: object

In [170]:
df_pkl[df_pkl["rawTime"] == 1140338843351]

,time,posX,posY,orientation,mac,signal,angle,rawTime,posXY


In [171]:
df_offline[df_offline["raw_time"] == 1139672815526]

,raw_time,pos_x,pos_y,degree,mac,rssi,angle,time,posXY
393941,1139672815526,9.0,4.0,135.1,00:14:bf:3b:c7:c6,-53,135,2006-02-11 15:46:55.526,9.0-4.0


In [172]:
for col in df_offline.columns:
    unique_vals = df_offline[col].unique()
    print(f"Column '{col}' has {len(unique_vals)} unique values:")

Column 'raw_time' has 146074 unique values:
Column 'pos_x' has 34 unique values:
Column 'pos_y' has 14 unique values:
Column 'degree' has 203 unique values:
Column 'mac' has 7 unique values:
Column 'rssi' has 73 unique values:
Column 'angle' has 8 unique values:
Column 'time' has 146074 unique values:
Column 'posXY' has 166 unique values:


In [173]:
for col in df_pkl.columns:
    unique_vals = df_pkl[col].unique()
    print(f"Column '{col}' has {len(unique_vals)} unique values:")

Column 'time' has 146041 unique values:
Column 'posX' has 34 unique values:
Column 'posY' has 14 unique values:
Column 'orientation' has 203 unique values:
Column 'mac' has 7 unique values:
Column 'signal' has 73 unique values:
Column 'angle' has 8 unique values:
Column 'rawTime' has 146041 unique values:
Column 'posXY' has 166 unique values:


In [204]:
df_offline[df_offline["raw_time"] == 1140339073150]

,raw_time,pos_x,pos_y,degree,mac,rssi,angle,time,posXY
645124,1140339073150,14.0,3.0,270.0,00:14:bf:3b:c7:c6,-54,270,2006-02-19 08:51:13.150,14.0-3.0


In [205]:
unique_df1 = df_offline['raw_time'].unique()
unique_df2 = df_pkl['rawTime'].unique()

diff_values = set(unique_df1) - set(unique_df2)
print("Values in df1 but not in df2:")
for val in diff_values:
    print(val, " - amount: ", len(df_offline[df_offline["raw_time"] == val]))

print(len(diff_values))

Values in df1 but not in df2:
1139673003138  - amount:  1
1140338968707  - amount:  1
1139672919685  - amount:  1
1139666646917  - amount:  1
1139672784144  - amount:  1
1141929121043  - amount:  1
1139671899161  - amount:  1
1139672867609  - amount:  1
1139666697120  - amount:  1
1139672992801  - amount:  1
1140339010466  - amount:  1
1139672909348  - amount:  1
1139672815526  - amount:  1
1140338885166  - amount:  1
1139671002045  - amount:  1
1139672690238  - amount:  1
1139672293950  - amount:  1
1141929100230  - amount:  1
1139666642632  - amount:  1
1139672731977  - amount:  1
1139672481481  - amount:  1
1139672836299  - amount:  1
1140338979148  - amount:  1
1140338926925  - amount:  1
1139673044945  - amount:  1
1141929152469  - amount:  1
1139666728406  - amount:  1
1140338843351  - amount:  1
1139672335704  - amount:  1
1140339052260  - amount:  1
1141929079397  - amount:  1
1140338812017  - amount:  1
1140339073150  - amount:  1
33


### Explorative Data Analysis

After the data was successfully loaded and transformed into a dataframe, the next step is to perform explorative data analysis on the dataframe.

In [ ]:
# TODO: implement

### Loading the online data

In [235]:
online_file = "data/online.final.trace.txt"
submacs = ["00:0f:a3:39:e1:c0", "00:0f:a3:39:dd:cd", "00:14:bf:b1:97:8a", "00:14:bf:3b:c7:c6", "00:14:bf:b1:97:90", "00:14:bf:b1:97:8d", "00:14:bf:b1:97:81"]

df_online = read_data(online_file, submacs)
df_online

,raw_time,pos_x,pos_y,degree,mac,rssi,angle,time,posXY
0,1139692477303,0.00,0.05,130.5,00:14:bf:b1:97:8a,-43,135,2006-02-11 21:14:37.303,0.0-0.05
1,1139692477303,0.00,0.05,130.5,00:0f:a3:39:e1:c0,-52,135,2006-02-11 21:14:37.303,0.0-0.05
2,1139692477303,0.00,0.05,130.5,00:14:bf:3b:c7:c6,-62,135,2006-02-11 21:14:37.303,0.0-0.05
3,1139692477303,0.00,0.05,130.5,00:14:bf:b1:97:81,-58,135,2006-02-11 21:14:37.303,0.0-0.05
4,1139692477303,0.00,0.05,130.5,00:14:bf:b1:97:8d,-62,135,2006-02-11 21:14:37.303,0.0-0.05
...,...,...,...,...,...,...,...,...,...
53297,1141928791755,21.45,6.62,27.3,00:14:bf:b1:97:8d,-43,45,2006-03-09 18:26:31.755,21.45-6.62
53298,1141928791755,21.45,6.62,27.3,00:0f:a3:39:e1:c0,-56,45,2006-03-09 18:26:31.755,21.45-6.62
53299,1141928791755,21.45,6.62,27.3,00:14:bf:b1:97:8a,-59,45,2006-03-09 18:26:31.755,21.45-6.62
53300,1141928791755,21.45,6.62,27.3,00:14:bf:3b:c7:c6,-59,45,2006-03-09 18:26:31.755,21.45-6.62
